# HABv4 SRK Table
In HAB4 it is possible to include up to four SRKs in a signed image, although only one may be used per reset cycle. By collecting SRKs in a table it is possible to select one of the SRKs at boot time. Any of the SRKs in the table may be selected without having to change the SRK_HASH value burned to efuses on the SoC.
This is useful on NXP processors where additional fuses are available for SRK revocation. That is, in the event one or more of the SRKs in the table are compromised, efuses corresponding to the compromised keys can be burned preventing those SRKs from ever being used again. This is enforced by the HAB library. The next SRK in the table can be used to sign new images. A minimum of one and maximum of four SRKs can be placed in an SRK table.

> **Note**:  Only the first three SRKs in a table can be revoked, so it recommended to use an SRK table with four keys in order to have one SRK to fall back on which cannot be revoked.

## 1. Prerequisites
- SPSDK is needed with examples extension. `pip install spsdk[examples]` (Please refer to the [installation](../../_knowledge_base/installation_guide.rst) documentation.)

In [1]:
import os

WORKSPACE = "workspace/"  # change this to path to your workspace
DATA_DIR = os.path.join("..", "..", "_data", "certificates")
SRK_CRTS = [
    "srk0_sha256_4096_65537_v3_usr_crt.pem",
    "srk1_sha256_4096_65537_v3_usr_crt.pem",
    "srk2_sha256_4096_65537_v3_usr_crt.pem",
    "srk3_sha256_4096_65537_v3_usr_crt.pem",
]

## 2. SRK Table generation

In [2]:
import os

from spsdk.crypto.certificate import Certificate
from spsdk.image.secret import SrkItem, SrkTable
from spsdk.utils.misc import write_file

# Create SRK Table instance
hab_srk = SrkTable(version=0x40)

for cert_file in SRK_CRTS:
    certificate = Certificate.load(os.path.join(DATA_DIR, cert_file))
    srk_item = SrkItem.from_certificate(certificate)
    hab_srk.append(srk_item)

print(hab_srk)

# Export binary SRK table
print("SRK table in binary:")
srk_binary = hab_srk.export()
print(srk_binary.hex())

srk_binary_path = os.path.join(WORKSPACE, "srk_table.bin")
print(f"SRK table saved to: {srk_binary_path}")
write_file(srk_binary, srk_binary_path, mode="wb")

srk_fuses_path = os.path.join(WORKSPACE, "srk_fuses.bin")
print(f"SRK hash fuses values exported to: {srk_fuses_path}")
write_file(hab_srk.export_fuses(), srk_fuses_path, "wb")

------------------------------------------------------------
SRK Table (Version: 4.0, #Keys: 4)
------------------------------------------------------------
SRK Key Index: 0 
Algorithm: EnumAlgorithm.PKCS1
Flag:      0x00 
Length:    4096 bit
Modulus:
    00:95:28:86:38:7b:e9:b6:4a:b8:88:cd:5b:22:41:
    2f:cb:da:63:74:0c:3c:ae:61:7a:32:cc:30:41:26:
    09:53:34:08:68:d4:46:35:32:9a:13:a6:81:81:9a:
    d9:b8:78:25:84:ef:27:f0:4e:28:d7:36:11:c8:e5:
    03:d1:69:f8:f4:c6:e0:7a:f0:98:88:89:01:be:57:
    f4:98:0b:24:e3:7d:7e:d4:27:fa:f7:4b:73:51:8c:
    31:bc:c8:e8:f4:43:16:ed:3c:b8:23:ee:fc:42:53:
    38:44:f9:b3:4b:51:db:8d:76:43:c0:53:6d:8a:09:
    33:a5:0c:2e:e1:3d:2d:ec:d5:c9:15:5f:ca:b6:53:
    b4:71:0f:30:4b:7d:f6:15:0b:fe:e9:90:79:0f:0d:
    e3:a4:02:94:67:bd:c9:65:f5:f0:a9:9e:3f:3b:e3:
    32:ec:8e:cd:ea:89:49:d7:22:97:f2:c3:93:69:6b:
    62:ef:6c:2a:65:78:2b:77:27:68:50:40:84:26:a5:
    86:18:f7:b4:d6:d5:93:85:66:de:23:db:04:c3:b5:
    cb:7f:a2:f9:37:05:e0:80:f2:b9:91:52:a4:5d:63

32